# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:

# Load CSV file generated from WeatherPy Folder
weather = "weather_df.csv"
weather1 = pd.read_csv(weather)
weather_data = weather1.dropna()
del weather_data["Unnamed: 0"]
weather_data.head()

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,pokhara,NP,2022-06-19 22:51:59,28.2333,83.9833,96.0,84.0,73.148,2.71
1,manzanillo,CU,2022-06-19 22:52:00,20.3433,-77.1167,100.0,78.0,76.190,0.14
2,rikitea,PF,2022-06-19 22:52:00,-23.1203,-134.9692,100.0,73.0,72.680,6.05
3,carnarvon,AU,2022-06-19 22:52:01,-24.8667,113.6333,1.0,70.0,68.594,5.59
4,hami,CN,2022-06-19 22:52:02,42.8000,93.4500,100.0,11.0,89.294,2.14


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = weather_data[["Latitude", "Longitude"]]

humidity = weather_data["Humidity (%)"].astype(float)

In [11]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
# Filter vacation with zero cloudiness
vacation_no_cloud = weather_data[weather_data["Cloudiness (%)"] == 0]
# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp (F)"] > 70]
# Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp (F)"] < 80]
# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed (mph)"] < 10]
# Filter vacation with humidity below 60 %
vacation = vacation_slow_wind[vacation_slow_wind["Humidity (%)"] < 60]
# Set Index
perfect_vacation = vacation.reset_index()
del perfect_vacation["index"]
perfect_vacation.dropna()
perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,karratha,AU,2022-06-19 22:53:40,-20.7377,116.8463,0.0,50.0,76.550,1.76
1,avera,US,2022-06-19 22:54:18,33.1940,-82.5271,0.0,31.0,72.824,6.32
2,russell,US,2022-06-19 22:54:30,32.3502,-85.1999,0.0,53.0,74.282,2.57
3,rakhiv,UA,2022-06-19 22:55:52,48.0526,24.2009,0.0,43.0,79.934,2.46
4,bocaiuva,BR,2022-06-19 22:56:27,-17.1078,-43.8150,0.0,29.0,73.382,5.29


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
# Hotel variable
hotel_df = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(perfect_vacation["City"])):

    lat = perfect_vacation.loc[city]["Latitude"]
    lng = perfect_vacation.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotel_df.append(hotel_response["results"][0]["name"])
    except:
        hotel_df.append("Nearest hotel not found")

perfect_vacation["Nearest Hotel"] = hotel_df
perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Nearest Hotel
0,karratha,AU,2022-06-19 22:53:40,-20.7377,116.8463,0.0,50.0,76.550,1.76,ibis Styles Karratha
1,avera,US,2022-06-19 22:54:18,33.1940,-82.5271,0.0,31.0,72.824,6.32,Nearest hotel not found
2,russell,US,2022-06-19 22:54:30,32.3502,-85.1999,0.0,53.0,74.282,2.57,Macs
3,rakhiv,UA,2022-06-19 22:55:52,48.0526,24.2009,0.0,43.0,79.934,2.46,Olenka
4,bocaiuva,BR,2022-06-19 22:56:27,-17.1078,-43.8150,0.0,29.0,73.382,5.29,Hotel Maxim's


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perfect_vacation.iterrows()]
locations = perfect_vacation[["Latitude", "Longitude"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map
# Display figure
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vaca_fig.add_layer(markers)

# Display Map
vaca_fig